In [3]:
pip install protobuf==3.20.3


Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset
import copy
import types
import weakref  # <--- NEW: Import weakref for non-registering parent reference
# --- START: Data Utilities Imports ---
import os
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import defaultdict
import math
# --- END: Data Utilities Imports ---

class MoEFFN(nn.Module):
    """
    Mixture-of-Experts Feed-Forward Network replacing the standard FFN in transformer layers.
    Supports load balancing loss.
    """
    def __init__(self, config, num_experts=8, top_k=2, load_balance_coeff=0.01):
        super().__init__()
        self.hidden_size = config.hidden_size
        self.intermediate_size = config.intermediate_size
        self.gate = nn.Linear(self.hidden_size, num_experts)
        self.experts = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.hidden_size, self.intermediate_size),
                nn.GELU(),
                nn.Linear(self.intermediate_size, self.hidden_size)
            ) for _ in range(num_experts)
        ])
        self.top_k = top_k
        self.num_experts = num_experts
        self.load_balance_coeff = load_balance_coeff

    def forward(self, hidden_states):
        batch_size, seq_len, hidden_size = hidden_states.shape
        flat_hidden = hidden_states.reshape(-1, hidden_size)  # (b*s, d)
        num_tokens = flat_hidden.shape[0]
        gate_logits = self.gate(flat_hidden)  # (b*s, e)
        gate_probs = torch.softmax(gate_logits, dim=-1)
        topk_probs, topk_idx = gate_probs.topk(self.top_k, dim=-1)  # (b*s, k)
        final_output = torch.zeros_like(flat_hidden)
        balance_loss = torch.zeros((), device=hidden_states.device)
        if self.training:
            dispatch_fraction = torch.zeros(self.num_experts, device=hidden_states.device)
            router_prob_mean = gate_probs.mean(dim=0)
        for e in range(self.num_experts):
            expert_mask = (topk_idx == e)
            token_mask = expert_mask.any(dim=-1)
            flat_token_indices = torch.nonzero(token_mask).squeeze(-1)
            if flat_token_indices.numel() == 0:
                continue
            input_e = flat_hidden.index_select(0, flat_token_indices)
            out_e = self.experts[e](input_e)
            probs_for_e = (topk_probs * expert_mask.float()).sum(dim=-1)
            weights = probs_for_e[token_mask]
            weighted = out_e * weights.unsqueeze(-1)
            final_output.index_add_(0, flat_token_indices, weighted)
            if self.training:
                dispatch_fraction[e] = token_mask.float().sum() / num_tokens
        if self.training:
            balance_loss = self.load_balance_coeff * self.num_experts * (dispatch_fraction * router_prob_mean).sum()
        return final_output.reshape(batch_size, seq_len, hidden_size), balance_loss

class Projection(nn.Module):
    """
    Lightweight MLP projection head for contrastive embeddings.
    """
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.act = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, out_dim)

    def forward(self, h):
        return self.fc2(self.act(self.fc1(h)))

class FedMoECR(nn.Module):
    """
    FedMoE-CR model: CodeBERT backbone with MoE layers and projection head.
    """
    def __init__(self, num_experts=8, top_k=2, proj_dim=128, load_balance_coeff=0.01):
        super().__init__()
        self.encoder = AutoModel.from_pretrained("microsoft/codebert-base")
        dim = self.encoder.config.hidden_size
        self.projection = Projection(dim, dim * 4, proj_dim)
        self.log_tau = nn.Parameter(torch.tensor(math.log(0.07)))
        self.total_balance_loss = torch.tensor(0.0)  # Initialized as tensor; reset in compute_loss
        # Replace two mid-layer FFN blocks with MoE
        for i in [5, 8]:  # Layers 6 and 9 (0-indexed)
            layer = self.encoder.encoder.layer[i]
            # 1. Attach MoE FFN and Parent Reference
            moe_ffn = MoEFFN(self.encoder.config, num_experts, top_k, load_balance_coeff)
            layer.moe_ffn = moe_ffn
            # CRITICAL FIX: Use weakref to store parent reference without module registration
            layer.parent_model = weakref.ref(self)
            # 2. Set up LayerNorm aliases (CodeBERT structure)
            layer.layernorm_before = layer.attention.output.LayerNorm
            layer.layernorm_after = layer.output.LayerNorm

            def new_forward(layer_self, hidden_states, attention_mask=None, head_mask=None,
                            encoder_hidden_states=None, encoder_attention_mask=None,
                            past_key_value=None, output_attentions=False):
                # 1. Self-Attention Block
                self_attention_outputs = layer_self.attention(
                    hidden_states, attention_mask, head_mask, output_attentions=output_attentions,
                )
                attention_output = self_attention_outputs[0]
                outputs = self_attention_outputs[1:]
                # No extra residual/LN here; attention_output is already post-residual + LN
                normalized_attention_output = attention_output
                # 3. MoE Feed-Forward Network
                moe_output, moe_balance_loss = layer_self.moe_ffn(normalized_attention_output)
                # 4. Second Residual Connection + LayerNorm (Post-MoE)
                moe_output = layer_self.output.dropout(moe_output)
                layer_output = layer_self.layernorm_after(moe_output + normalized_attention_output)
                # 5. Accumulate Loss
                if layer_self.training:
                    # CRITICAL FIX: Dereference weakref and check if parent is alive
                    parent = layer_self.parent_model()
                    if parent is not None:
                        parent.total_balance_loss += moe_balance_loss
                outputs = (layer_output,) + outputs
                return outputs

            # 3. Bind the new forward method
            layer.forward = types.MethodType(new_forward, layer)

    def forward(self, inputs):
        # No reset here; handled in compute_loss to accumulate across query/code forwards
        outputs = self.encoder(**inputs)
        h = outputs.last_hidden_state[:, 0]  # CLS token
        z = self.projection(h)
        z = F.normalize(z, dim=1)
        return z

class CodeDataset(Dataset):
    """
    Dataset for query-code pairs from CodeSearchNet.
    Modified to take a Pandas DataFrame as input.
    """
    def __init__(self, df):
        self.queries = df['docstring'].tolist()
        self.codes = df['code'].tolist()

    def __len__(self):
        return len(self.queries)

    def __getitem__(self, idx):
        return self.queries[idx], self.codes[idx]

def collate_fn(batch, tokenizer):
    queries, codes = zip(*batch)
    q_inputs = tokenizer(list(queries), padding=True, truncation=True, return_tensors='pt', max_length=128)
    c_inputs = tokenizer(list(codes), padding=True, truncation=True, return_tensors='pt', max_length=512)
    return q_inputs, c_inputs

def compute_loss(model, q_inputs, c_inputs, device):
    q_inputs = {k: v.to(device) for k, v in q_inputs.items()}
    c_inputs = {k: v.to(device) for k, v in c_inputs.items()}
    if model.training:
        model.total_balance_loss = torch.zeros((), device=device)
    z_q = model(q_inputs)
    z_c = model(c_inputs)
    tau = model.log_tau.exp()
    dots = torch.mm(z_q, z_c.T)
    sim = dots / tau
    labels = torch.arange(z_q.size(0)).to(device)
    nce_loss = nn.CrossEntropyLoss()(sim, labels)
    total_loss = nce_loss
    if model.training:
        total_loss += model.total_balance_loss
    return total_loss

def apply_dp_gradients(model, clip_norm, noise_std, batch_size):
    """
    Apply gradient clipping and noise for differential privacy.
    Approximates DP by scaling noise appropriately (note: this is batch-level, not per-sample DP).
    """
    params = [p for p in model.parameters() if p.grad is not None]
    if len(params) == 0:
        return
    total_norm = torch.sqrt(sum(p.grad.norm(2)**2 for p in params))
    if total_norm > clip_norm:
        for p in params:
            p.grad.detach().mul_(clip_norm / total_norm)
    effective_std = noise_std * clip_norm / batch_size  # Scale for approximate DP
    for p in params:
        noise = torch.normal(
            mean=0.0,
            std=effective_std,
            size=p.grad.shape,
            device=p.grad.device,
            dtype=p.grad.dtype
        )
        p.grad += noise

def local_update(model, dataloader, optimizer, device, local_steps, clip_norm, noise_std):
    model.train()
    for _ in range(local_steps):
        for q_inputs, c_inputs in dataloader:
            batch_size = q_inputs['input_ids'].shape[0]  # Get current batch size
            optimizer.zero_grad()
            loss = compute_loss(model, q_inputs, c_inputs, device)
            loss.backward()
            apply_dp_gradients(model, clip_norm, noise_std, batch_size)
            optimizer.step()

def federated_aggregation(global_model, client_models, client_sizes):
    total_size = sum(client_sizes)
    new_state_dict = {}
    for key in global_model.state_dict().keys():
        new_state_dict[key] = torch.zeros_like(global_model.state_dict()[key])
        for i, size in enumerate(client_sizes):
            new_state_dict[key] += (size / total_size) * client_models[i].state_dict()[key]
    global_model.load_state_dict(new_state_dict)

# ----------------------------------------------------------------------
## 💻 Data Preparation Utilities (for Non-IID Sharding)
# ----------------------------------------------------------------------
def clean_df(df, max_ds_len=300, max_code_len=800):
    """Keep only rows with non-null, reasonably short docstring & code."""
    df = df[['docstring', 'code', 'path']].dropna()
    df = df[df['docstring'].str.len() < max_ds_len]
    df = df[df['code'].str.len() < max_code_len]
    return df.reset_index(drop=True)

def shard_by_owner(df, K):
    """
    Simulate K clients by grouping functions by 'owner' (first part of path).
    Returns a list of K DataFrames, each a heterogeneous shard.
    """
    df['owner'] = df['path'].apply(lambda p: os.path.normpath(p).split(os.sep)[0])
    owners = df['owner'].unique().tolist()
    np.random.shuffle(owners)
    shards = defaultdict(list)
    for idx, owner in enumerate(owners):
        shard_id = idx % K
        shards[shard_id].append(owner)
    shard_dfs = []
    for k in range(K):
        owners_k = shards[k]
        df_k = df[df['owner'].isin(owners_k)].drop(columns=['owner'])
        shard_dfs.append(df_k.reset_index(drop=True))
    return shard_dfs

def load_and_preprocess(lang_dir, K=5, val_frac=0.05, random_state=42, percentage=1.0):
    """
    Load and preprocess CodeSearchNet data for one language.
    """
    assert 0 < percentage <= 1.0, "percentage must be in (0, 1]"
    train_files = glob.glob(os.path.join(lang_dir, 'final/jsonl/train/*.jsonl'))
    test_files = glob.glob(os.path.join(lang_dir, 'final/jsonl/test/*.jsonl'))
    train_df = pd.concat([pd.read_json(f, lines=True) for f in train_files], ignore_index=True)
    test_df = pd.concat([pd.read_json(f, lines=True) for f in test_files], ignore_index=True)
    if percentage < 1.0:
        train_df = train_df.sample(frac=percentage, random_state=random_state).reset_index(drop=True)
        test_df = test_df.sample(frac=percentage, random_state=random_state).reset_index(drop=True)
    train_df = clean_df(train_df)
    test_df = clean_df(test_df)
    train_df, public_val = train_test_split(
        train_df, test_size=val_frac, random_state=random_state, shuffle=True
    )
    client_shards = shard_by_owner(train_df, K)
    return {
        'clients': client_shards,
        'public_val': public_val,
        'test': test_df
    }

def prepare_all_languages(base_dir, languages, K=5, percentage=1.0):
    """
    Iterate over each language directory, preprocess, and collect.
    """
    all_data = {}
    for lang in languages:
        lang_path = os.path.join(base_dir, lang, lang)
        if not os.path.isdir(lang_path):
            print(f"⚠️ Skipping '{lang_path}' – directory not found.")
            continue
        print(f"⏳ Preprocessing {lang}...")
        all_data[lang] = load_and_preprocess(lang_path, K=K, percentage=percentage)
        print(f"✅ Done: {lang} -> {len(all_data[lang]['clients'])} shards ({sum(len(df) for df in all_data[lang]['clients'])} total samples).")
    return all_data

def flatten_client_shards(federated_data):
    """
    Combines the client lists from all languages into a single list of client DataFrames.
    """
    all_clients = []
    for lang, data in federated_data.items():
        all_clients.extend(data['clients'])
    return all_clients

# ----------------------------------------------------------------------
## 🚀 Main Training Script
# ----------------------------------------------------------------------
# Hyperparameters
NUM_EXPERTS = 8
TOP_K = 2
PROJ_DIM = 128
BATCH_SIZE = 16
LOCAL_STEPS = 1
LEARNING_RATE = 1e-5
CLIP_NORM = 1.0
NOISE_STD = 0.01
NUM_ROUNDS = 2
LANGUAGES = ['python', 'java', 'go']
LOAD_BALANCE_COEFF = 0.01
NUM_CLIENTS_PER_LANG = 1  # K
DATA_LOAD_PERCENTAGE = 0.001 # <--- Set to 0.001 for a rapid test (0.1% or 1/1000th)

# Main training script
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
    global_model = FedMoECR(NUM_EXPERTS, TOP_K, PROJ_DIM, LOAD_BALANCE_COEFF).to(device)
    ## 🌐 Data Loading & Sharding Logic
    print("\n--- Starting Federated Data Preparation ---")
    BASE_DIR = "/kaggle/input/codesearchnet"
    federated_data = prepare_all_languages(BASE_DIR, LANGUAGES, K=NUM_CLIENTS_PER_LANG, percentage=DATA_LOAD_PERCENTAGE)
    all_client_dataframes = flatten_client_shards(federated_data)
    print(f"Total number of clients for FL: {len(all_client_dataframes)}\n")
    client_dataloaders = []
    client_sizes = []
    for df in all_client_dataframes:
        if len(df) > 0:
            code_ds = CodeDataset(df)
            client_sizes.append(len(code_ds))
            dl = DataLoader(code_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=lambda b: collate_fn(b, tokenizer))
            client_dataloaders.append(dl)
        else:
            print("Skipped an empty client shard.")
    ## 🚀 START TRAINING
    print("--- Starting Federated Training Rounds ---")
    for round_num in range(NUM_ROUNDS):
        print(f"Round {round_num + 1}/{NUM_ROUNDS}")
        client_models = []
        for dl in client_dataloaders:
            client_model = FedMoECR(NUM_EXPERTS, TOP_K, PROJ_DIM, LOAD_BALANCE_COEFF).to(device)
            client_model.load_state_dict(global_model.state_dict())
            optimizer = optim.Adam(client_model.parameters(), lr=LEARNING_RATE)
            local_update(client_model, dl, optimizer, device, LOCAL_STEPS, CLIP_NORM, NOISE_STD)
            client_models.append(client_model)
        federated_aggregation(global_model, client_models, client_sizes)
    torch.save(global_model.state_dict(), "fedmoe_cr_model.pth")
    # Example Inference
    def infer_code_recommendation(model, tokenizer, query, candidate_codes, device, top_k=5):
        model.eval()
        with torch.no_grad():
            q_inputs = tokenizer(query, return_tensors='pt', truncation=True, max_length=128).to(device)
            z_q = model(q_inputs)
            z_cs = []
            for code in candidate_codes:
                c_inputs = tokenizer(code, return_tensors='pt', truncation=True, max_length=512).to(device)
                z_c = model(c_inputs)
                z_cs.append(z_c)
            z_cs = torch.cat(z_cs, dim=0)
            scores = F.cosine_similarity(z_q, z_cs)
            top_indices = scores.topk(top_k).indices
            return [candidate_codes[i] for i in top_indices]
    query = "Sort a list in Python"
    candidates = ["def sort_list(l): return sorted(l)", "l.sort()", "import numpy as np; np.sort(l)", "random code"]
    recommendations = infer_code_recommendation(global_model, tokenizer, query, candidates, device)
    print("\nTop recommendations:", recommendations)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

2025-12-04 05:07:17.780737: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764824838.038429      38 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764824838.108538      38 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]


--- Starting Federated Data Preparation ---
⏳ Preprocessing python...
✅ Done: python -> 1 shards (209 total samples).
⏳ Preprocessing java...
✅ Done: java -> 1 shards (244 total samples).
⏳ Preprocessing go...
✅ Done: go -> 1 shards (245 total samples).
Total number of clients for FL: 3

--- Starting Federated Training Rounds ---
Round 1/2
Round 2/2


RuntimeError: selected index k out of range

In [6]:
query = "Sort a list in Python"
candidates = [
    "def sort_list(l): return sorted(l)",
    "l.sort()",
    "import numpy as np; np.sort(l)",
    "random code",
    "another random code block" # <--- Added 5th candidate
]
recommendations = infer_code_recommendation(global_model, tokenizer, query, candidates, device)

In [7]:
print("\nTop recommendations:", recommendations)


Top recommendations: ['another random code block', 'l.sort()', 'random code', 'import numpy as np; np.sort(l)', 'def sort_list(l): return sorted(l)']


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset
import copy
import types
import weakref
import os
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import defaultdict
import math
import matplotlib.pyplot as plt # <--- NEW: Import for plotting
# --- END: Data Utilities Imports ---

# ----------------------------------------------------------------------
## 📊 NEW: MoE Diagnostics Class
# ----------------------------------------------------------------------
class MoEDiagnostics:
    """Stores and accumulates expert dispatch statistics across a forward pass."""
    def __init__(self, num_experts, num_layers):
        self.num_experts = num_experts
        self.num_layers = num_layers
        self.reset()

    def reset(self):
        # Accumulated dispatch fraction for each expert across all MoE layers
        self.expert_dispatch_sum = torch.zeros(self.num_layers, self.num_experts)
        self.sample_count = 0

    def record(self, layer_idx, dispatch_fraction):
        """Records the dispatch fraction (load) for a single layer/batch."""
        self.expert_dispatch_sum[layer_idx] += dispatch_fraction.cpu()
        
    def get_avg_dispatch(self):
        """Calculates the average dispatch fraction per expert."""
        # Note: If called after a full forward pass, sample_count should be updated externally
        if self.sample_count == 0:
            return torch.zeros(self.num_layers, self.num_experts)
        return self.expert_dispatch_sum / self.sample_count


class MoEFFN(nn.Module):
    """
    Mixture-of-Experts Feed-Forward Network replacing the standard FFN in transformer layers.
    Supports load balancing loss and records dispatch diagnostics.
    """
    def __init__(self, config, num_experts=8, top_k=2, load_balance_coeff=0.01, layer_idx=None): # <--- NEW: layer_idx
        super().__init__()
        self.hidden_size = config.hidden_size
        self.intermediate_size = config.intermediate_size
        self.gate = nn.Linear(self.hidden_size, num_experts)
        self.experts = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.hidden_size, self.intermediate_size),
                nn.GELU(),
                nn.Linear(self.intermediate_size, self.hidden_size)
            ) for _ in range(num_experts)
        ])
        self.top_k = top_k
        self.num_experts = num_experts
        self.load_balance_coeff = load_balance_coeff
        self.layer_idx = layer_idx # <--- NEW: Store layer index for diagnostics
        self.parent_diagnostics = None # <--- NEW: Placeholder for Diagnostics object

    def forward(self, hidden_states):
        batch_size, seq_len, hidden_size = hidden_states.shape
        flat_hidden = hidden_states.reshape(-1, hidden_size)  # (b*s, d)
        num_tokens = flat_hidden.shape[0]
        gate_logits = self.gate(flat_hidden)  # (b*s, e)
        gate_probs = torch.softmax(gate_logits, dim=-1)
        topk_probs, topk_idx = gate_probs.topk(self.top_k, dim=-1)  # (b*s, k)
        final_output = torch.zeros_like(flat_hidden)
        balance_loss = torch.zeros((), device=hidden_states.device)
        
        # --- NEW: Dispatch tracking initialization ---
        dispatch_fraction = None
        router_prob_mean = None
        if self.training or (self.parent_diagnostics is not None and not self.training):
            dispatch_fraction = torch.zeros(self.num_experts, device=hidden_states.device)
            router_prob_mean = gate_probs.mean(dim=0)
        # ---------------------------------------------

        for e in range(self.num_experts):
            expert_mask = (topk_idx == e)
            token_mask = expert_mask.any(dim=-1)
            flat_token_indices = torch.nonzero(token_mask).squeeze(-1)
            if flat_token_indices.numel() == 0:
                continue
            input_e = flat_hidden.index_select(0, flat_token_indices)
            out_e = self.experts[e](input_e)
            probs_for_e = (topk_probs * expert_mask.float()).sum(dim=-1)
            weights = probs_for_e[token_mask]
            weighted = out_e * weights.unsqueeze(-1)
            final_output.index_add_(0, flat_token_indices, weighted)
            
            if dispatch_fraction is not None:
                dispatch_fraction[e] = token_mask.float().sum() / num_tokens
        
        # --- NEW: Record diagnostics on both train and eval ---
        if self.parent_diagnostics is not None and dispatch_fraction is not None:
            self.parent_diagnostics.record(self.layer_idx, dispatch_fraction)
        # ----------------------------------------------------
        
        if self.training:
            if dispatch_fraction is not None and router_prob_mean is not None:
                balance_loss = self.load_balance_coeff * self.num_experts * (dispatch_fraction * router_prob_mean).sum()
        
        return final_output.reshape(batch_size, seq_len, hidden_size), balance_loss

class Projection(nn.Module):
    # ... (Projection class remains unchanged)
    # ...
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.act = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, out_dim)

    def forward(self, h):
        return self.fc2(self.act(self.fc1(h)))


class FedMoECR(nn.Module):
    """
    FedMoE-CR model: CodeBERT backbone with MoE layers and projection head.
    """
    def __init__(self, num_experts=8, top_k=2, proj_dim=128, load_balance_coeff=0.01):
        super().__init__()
        self.encoder = AutoModel.from_pretrained("microsoft/codebert-base")
        dim = self.encoder.config.hidden_size
        self.projection = Projection(dim, dim * 4, proj_dim)
        self.log_tau = nn.Parameter(torch.tensor(math.log(0.07)))
        self.total_balance_loss = torch.tensor(0.0)
        
        # --- NEW: MoE Layer Indices & Diagnostics Setup ---
        self.moe_layer_indices = [5, 8] # Layers 6 and 9
        self.diagnostics = MoEDiagnostics(num_experts, len(self.moe_layer_indices)) # <--- NEW: Diagnostics object
        # ---------------------------------------------------
        
        # Replace two mid-layer FFN blocks with MoE
        for moe_idx, i in enumerate(self.moe_layer_indices): # <--- NEW: Enumerate for layer index
            layer = self.encoder.encoder.layer[i]
            # 1. Attach MoE FFN and Parent Reference
            moe_ffn = MoEFFN(self.encoder.config, num_experts, top_k, load_balance_coeff, layer_idx=moe_idx) # <--- Pass moe_idx
            moe_ffn.parent_diagnostics = self.diagnostics # <--- NEW: Link diagnostics object
            layer.moe_ffn = moe_ffn
            # CRITICAL FIX: Use weakref to store parent reference without module registration
            layer.parent_model = weakref.ref(self)
            # 2. Set up LayerNorm aliases (CodeBERT structure)
            layer.layernorm_before = layer.attention.output.LayerNorm
            layer.layernorm_after = layer.output.LayerNorm

            # ... (new_forward remains the same)
            def new_forward(layer_self, hidden_states, attention_mask=None, head_mask=None,
                            encoder_hidden_states=None, encoder_attention_mask=None,
                            past_key_value=None, output_attentions=False):
                # 1. Self-Attention Block
                self_attention_outputs = layer_self.attention(
                    hidden_states, attention_mask, head_mask, output_attentions=output_attentions,
                )
                attention_output = self_attention_outputs[0]
                outputs = self_attention_outputs[1:]
                # No extra residual/LN here; attention_output is already post-residual + LN
                normalized_attention_output = attention_output
                # 3. MoE Feed-Forward Network
                moe_output, moe_balance_loss = layer_self.moe_ffn(normalized_attention_output)
                # 4. Second Residual Connection + LayerNorm (Post-MoE)
                moe_output = layer_self.output.dropout(moe_output)
                layer_output = layer_self.layernorm_after(moe_output + normalized_attention_output)
                # 5. Accumulate Loss
                if layer_self.training:
                    # CRITICAL FIX: Dereference weakref and check if parent is alive
                    parent = layer_self.parent_model()
                    if parent is not None:
                        parent.total_balance_loss += moe_balance_loss
                outputs = (layer_output,) + outputs
                return outputs

            # 3. Bind the new forward method
            layer.forward = types.MethodType(new_forward, layer)

    def forward(self, inputs):
        # Reset sample count here for diagnostics accumulation
        if not self.training:
             self.diagnostics.sample_count = 0
             self.diagnostics.expert_dispatch_sum = torch.zeros(self.diagnostics.num_layers, self.diagnostics.num_experts)

        outputs = self.encoder(**inputs)
        
        # Increment sample count for diagnostics
        if not self.training:
            # Assuming batch size is the first dimension of input_ids
            batch_size = inputs['input_ids'].shape[0]
            self.diagnostics.sample_count += batch_size
        
        h = outputs.last_hidden_state[:, 0]  # CLS token
        z = self.projection(h)
        z = F.normalize(z, dim=1)
        return z

# ... (CodeDataset, collate_fn, compute_loss, apply_dp_gradients, local_update, federated_aggregation remain mostly unchanged) ...


# ----------------------------------------------------------------------
## 📊 NEW: Evaluation and Graph Functions
# ----------------------------------------------------------------------

def calculate_metrics(z_q, z_c, device, top_ks=[1, 5, 10]):
    """Calculates R@K and MRR for a batch."""
    dots = torch.mm(z_q, z_c.T)
    # The correct code is on the diagonal
    labels = torch.arange(z_q.size(0)).to(device)
    
    # Sort by similarity score in descending order
    _, sorted_indices = dots.sort(dim=1, descending=True)
    
    # Calculate Rank and MRR
    # Find the position of the correct label in the sorted list (rank is 1-indexed)
    correct_ranks = (sorted_indices == labels.unsqueeze(1)).nonzero(as_tuple=True)[1] + 1
    mrr = (1.0 / correct_ranks.float()).sum().item()
    
    # Calculate Recall@K
    recall = {}
    for k in top_ks:
        # Check if the correct rank is less than or equal to k
        hits_at_k = (correct_ranks <= k).sum().item()
        recall[f'R@{k}'] = hits_at_k

    return mrr, recall


def evaluate_model(model, df_test, tokenizer, device, top_ks=[1, 5, 10]):
    """Evaluates the global model on the aggregated test set."""
    model.eval()
    test_ds = CodeDataset(df_test)
    test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=lambda b: collate_fn(b, tokenizer))

    total_mrr = 0
    total_samples = 0
    total_recall = {f'R@{k}': 0 for k in top_ks}

    with torch.no_grad():
        for q_inputs, c_inputs in test_dl:
            q_inputs = {k: v.to(device) for k, v in q_inputs.items()}
            c_inputs = {k: v.to(device) for k, v in c_inputs.items()}
            
            z_q = model(q_inputs)
            z_c = model(c_inputs)
            
            mrr, recall = calculate_metrics(z_q, z_c, device, top_ks)
            
            batch_size = z_q.size(0)
            total_mrr += mrr
            for k in top_ks:
                total_recall[f'R@{k}'] += recall[f'R@{k}']
            total_samples += batch_size
    
    avg_mrr = total_mrr / total_samples
    avg_recall = {k: v / total_samples for k, v in total_recall.items()}
    
    # After evaluation, calculate and return average expert dispatch
    avg_dispatch = model.diagnostics.get_avg_dispatch()

    return avg_mrr, avg_recall, avg_dispatch


def plot_expert_load_distribution(avg_dispatch, round_num, moe_layer_indices):
    """Plots the average dispatch fraction for all experts in all layers."""
    num_layers, num_experts = avg_dispatch.shape
    fig, axes = plt.subplots(1, num_layers, figsize=(num_experts * 2.5, 5), sharey=True)
    
    if num_layers == 1: # Handle case with only one layer
        axes = [axes]

    for i in range(num_layers):
        ax = axes[i]
        dispatch_data = avg_dispatch[i].numpy()
        ax.bar(range(num_experts), dispatch_data)
        ax.axhline(y=TOP_K / num_experts, color='r', linestyle='--', label=f'Target Load ({TOP_K}/{num_experts})')
        ax.set_title(f'MoE Layer {moe_layer_indices[i]}')
        ax.set_xticks(range(num_experts))
        ax.set_xlabel('Expert ID')
        ax.grid(axis='y', alpha=0.5)
        
    axes[0].set_ylabel('Average Dispatch Fraction')
    fig.suptitle(f'Expert Load Distribution (Round {round_num})')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    # Placeholder for saving/displaying the plot
    plt.savefig(f'expert_load_round_{round_num}.png')
    plt.close()
    print(f"Graph saved: expert_load_round_{round_num}.png ")


def plot_training_curve(results):
    """Plots the R@1 score over training rounds."""
    rounds = [r['round'] for r in results]
    r1_scores = [r['R@1'] * 100 for r in results] # Convert to percentage
    
    plt.figure(figsize=(10, 6))
    plt.plot(rounds, r1_scores, marker='o', label='Global Model R@1')
    plt.xlabel('Federated Round')
    plt.ylabel('Recall@1 (%)')
    plt.title('Code Recommendation Performance over FL Rounds')
    plt.grid(True)
    plt.legend()
    # Placeholder for saving/displaying the plot
    plt.savefig('performance_curve.png')
    plt.close()
    print("Graph saved: performance_curve.png ")

# ----------------------------------------------------------------------
## 🚀 Main Training Script (UPDATED)
# ----------------------------------------------------------------------
# Hyperparameters
NUM_EXPERTS = 8
TOP_K = 2
PROJ_DIM = 128
BATCH_SIZE = 16
LOCAL_STEPS = 1
LEARNING_RATE = 1e-5
CLIP_NORM = 1.0
NOISE_STD = 0.01
NUM_ROUNDS = 2 # Increase this for real research
LANGUAGES = ['python', 'java', 'go']
LOAD_BALANCE_COEFF = 0.01
NUM_CLIENTS_PER_LANG = 1
DATA_LOAD_PERCENTAGE = 0.001 

# Global results storage for plotting
GLOBAL_RESULTS = []

if __name__ == "__main__":
    # ... (Initialization code remains the same) ...
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
    global_model = FedMoECR(NUM_EXPERTS, TOP_K, PROJ_DIM, LOAD_BALANCE_COEFF).to(device)
    
    ## 🌐 Data Loading & Sharding Logic
    print("\n--- Starting Federated Data Preparation ---")
    BASE_DIR = "/kaggle/input/codesearchnet"
    federated_data = prepare_all_languages(BASE_DIR, LANGUAGES, K=NUM_CLIENTS_PER_LANG, percentage=DATA_LOAD_PERCENTAGE)
    all_client_dataframes = flatten_client_shards(federated_data)
    
    # Combine all test sets for global evaluation (This is crucial)
    aggregated_test_df = pd.concat([data['test'] for data in federated_data.values()], ignore_index=True)
    print(f"Total test samples for evaluation: {len(aggregated_test_df)}")

    print(f"Total number of clients for FL: {len(all_client_dataframes)}\n")
    client_dataloaders = []
    client_sizes = []
    for df in all_client_dataframes:
        if len(df) > 0:
            code_ds = CodeDataset(df)
            client_sizes.append(len(code_ds))
            dl = DataLoader(code_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=lambda b: collate_fn(b, tokenizer))
            client_dataloaders.append(dl)
        else:
            print("Skipped an empty client shard.")

    ## 🚀 START TRAINING
    print("--- Starting Federated Training Rounds ---")
    for round_num in range(NUM_ROUNDS):
        print(f"Round {round_num + 1}/{NUM_ROUNDS}")
        client_models = []
        
        # 1. Local Training
        for dl in client_dataloaders:
            # IMPORTANT: Re-initialize FedMoECR to ensure MoE_FFN is correctly linked to diagnostics
            client_model = FedMoECR(NUM_EXPERTS, TOP_K, PROJ_DIM, LOAD_BALANCE_COEFF).to(device)
            client_model.load_state_dict(global_model.state_dict())
            optimizer = optim.Adam(client_model.parameters(), lr=LEARNING_RATE)
            local_update(client_model, dl, optimizer, device, LOCAL_STEPS, CLIP_NORM, NOISE_STD)
            client_models.append(client_model)
            
        # 2. Aggregation
        federated_aggregation(global_model, client_models, client_sizes)
        
        # 3. Global Evaluation and Diagnostics (REQUIRED FOR GRAPHS)
        avg_mrr, avg_recall, avg_dispatch = evaluate_model(global_model, aggregated_test_df, tokenizer, device, top_ks=[1, 5, 10])
        
        print(f"--- Evaluation Round {round_num + 1} ---")
        print(f"MRR: {avg_mrr:.4f} | R@1: {avg_recall['R@1']:.4f} | R@5: {avg_recall['R@5']:.4f}")
        
        # Store results for plotting the learning curve
        GLOBAL_RESULTS.append({
            'round': round_num + 1,
            'MRR': avg_mrr,
            'R@1': avg_recall['R@1'],
            'avg_dispatch': avg_dispatch # Store for later analysis if needed
        })
        
        # Plot the expert load distribution (MoE Diagnostic Graph)
        plot_expert_load_distribution(avg_dispatch, round_num + 1, global_model.moe_layer_indices)

    # 4. Final steps
    torch.save(global_model.state_dict(), "fedmoe_cr_model.pth")
    
    # Plot the final performance curve
    plot_training_curve(GLOBAL_RESULTS)
    
    # Example Inference (keep this for code completion)
    # ... (infer_code_recommendation function call remains the same) ...
    def infer_code_recommendation(model, tokenizer, query, candidate_codes, device, top_k=5):
        model.eval()
        with torch.no_grad():
            q_inputs = tokenizer(query, return_tensors='pt', truncation=True, max_length=128).to(device)
            z_q = model(q_inputs)
            z_cs = []
            for code in candidate_codes:
                c_inputs = tokenizer(code, return_tensors='pt', truncation=True, max_length=512).to(device)
                z_c = model(c_inputs)
                z_cs.append(z_c)
            z_cs = torch.cat(z_cs, dim=0)
            scores = F.cosine_similarity(z_q, z_cs)
            top_indices = scores.topk(top_k).indices
            return [candidate_codes[i] for i in top_indices]
    
    query = "Sort a list in Python"
    candidates = ["def sort_list(l): return sorted(l)", "l.sort()", "import numpy as np; np.sort(l)", "random code", "another code"] # Added a fifth candidate
    recommendations = infer_code_recommendation(global_model, tokenizer, query, candidates, device, top_k=4) # Adjusted top_k
    print("\nTop recommendations:", recommendations)


--- Starting Federated Data Preparation ---
⏳ Preprocessing python...
✅ Done: python -> 1 shards (209 total samples).
⏳ Preprocessing java...
✅ Done: java -> 1 shards (244 total samples).
⏳ Preprocessing go...
✅ Done: go -> 1 shards (245 total samples).
Total test samples for evaluation: 42
Total number of clients for FL: 3

--- Starting Federated Training Rounds ---
Round 1/2
--- Evaluation Round 1 ---
MRR: 0.3023 | R@1: 0.1429 | R@5: 0.4524
Graph saved: expert_load_round_1.png 
Round 2/2
--- Evaluation Round 2 ---
MRR: 0.3311 | R@1: 0.1667 | R@5: 0.4524
Graph saved: expert_load_round_2.png 
Graph saved: performance_curve.png 

Top recommendations: ['l.sort()', 'another code', 'random code', 'def sort_list(l): return sorted(l)']
